<a href="https://colab.research.google.com/github/sanzano/sentencias/blob/main/NLP_ETL_Sentencias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-docx
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.2 MB/s eta 0:00:00


In [ ]:
!pip install pygsheets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 1.6 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import drive
from google.colab import files
from google.colab import auth
from oauth2client.service_account import ServiceAccountCredentials

import gspread


from PyPDF2 import PdfReader
import PyPDF2
import re
import pandas as pd

In [ ]:
drive.mount('/content/drive')

In [ ]:
extracted_path = '/content/drive/My Drive/INGENIERIA/0. Smtre 2024-2/NLP/PROYECTO/Sentencias/'

Se lee el excel con los datos de la página del TFJFA

In [ ]:
print(len(os.listdir(extracted_path)))

# Expresiones regulares


In [ ]:
#Expresiones regulares
cdmxpattern = re.compile(r'^.{0,15}(Ciudad de México)|(Distrito Federal).*?(\.en los términos siguientes:.*|$)', re.DOTALL)

expe_pattern = re.compile(r"(expe)|(acción\s+de\s+inconstitucionalidad\s+)", re.IGNORECASE)

actor_pattern = re.compile(r"(ACTOR)|(solici)|(nunciante)|(recurrent)|(promovent)|(INCIDENTISTA)", re.IGNORECASE)

terc_pattern = re.compile(r"(tercero?)|(interesad)", re.IGNORECASE)



#TEPJF_pattern = re.compile(r".*?(?=\d)", re.IGNORECASE)
#Tema_pattern = re.compile(r"Tema", re.IGNORECASE)
#Tipo_asunto_pattern = re.compile(r"Tipo de asun", re.IGNORECASE)
resp_pattern = re.compile(r"(resp)|(nunciad)|(demandad)|(señalad)", re.IGNORECASE)

pnte_pattern = re.compile(r"(magist)|(ponente)", re.IGNORECASE)
secr_pattern = re.compile(r"secret", re.IGNORECASE)

patternResultando = re.compile(r'(^.*R\s*E\s*S\s*U\s*L\s*T\s*A\s*N\s*D\s*O.*)')

patternConsiderando = re.compile(r'(^.*C\s*O\s*N\s*S\s*I\s*D\s*E\s*R\s*A\s*N\s*D\s*O.*)')

patternResuelve = re.compile(r'(^.*A\s*C\s*U\s*E\s*R\s*D.*$)|(^.*A\s*C\s*O\s*R\s*D.*)|(^.*R\s*E\s*S\s*U\s*E\s*L\s*V.*)|(.*R\s*E\s*S\s*O\s*L\s*U\s*T\s*I\s*V.*)|(.*A\s*C\s*U\s*E\s*R\s*D\s*A.*)')

patternNotif = re.compile(r'(^\s*N\s*o\s*t\s*i\s*f\s*[ií]\s*q\s*u\s*e\s*s\s*e.*?)|(^\s*N\s*o\s*t\s*i\s*f\s*i\s*c\s*a\s*r.*?)(?=\.|VISTO|$)', re.IGNORECASE)

# Definición de Funciones

In [ ]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PdfReader(file)
        num_pages = len(reader.pages)
        for page_number in range(num_pages):
            page = reader.pages[page_number]
            page_text = page.extract_text()
            # Replace consecutive newlines with a space
            #page_text = re.sub(r'\n{2,}', ' ', page_text)
            text += page_text + "\n"  # Add a newline after each page
            content = [paragraph for paragraph in text]
            result = ''.join(content)
    return text

def imprimirLista(lista):
  for index,text in enumerate(lista):
    print("Index:", index, "Text:", text)

In [ ]:
# Dividir el texto para extraer secciones específicas utilizando regex
def obtenerRubro(patternFin,parrafos):
  matching_indexFin = next((index for index, string in enumerate(parrafos) if re.search(patternFin, string)), None)
  if matching_indexFin is not None:
     #print(f"Index: {matching_indexFin}, Content: {parrafos[matching_indexFin]}")
     content = [paragraph for paragraph in parrafos[:matching_indexFin]]
            # Join the cleaned content into a single string
     #result_string = '\n'.join(content)
     return content
  else:
     return [0,0,0]

def obtenerVotacion(patternIni,parrafos):
  matching_indexIni = next((index for index, string in enumerate(parrafos) if re.search(patternIni, string)), None)
  if matching_indexIni is not None:
     content = [paragraph for paragraph in parrafos[matching_indexIni:]]
     return content
  else:
     return [0,0,0]

In [ ]:
def convertirADicc(rubro):
    diccionario = {}
    current_key = None
    for item in rubro:
      if isinstance(item, str) and ':' in item:
          parts = item.split(':')
          current_key = parts[0].strip().replace(" ","")
          value = parts[1].strip() if len(parts) > 1 else None
          diccionario[current_key] = value
      elif current_key is not None:
          diccionario[current_key] += f' {item.strip()}'
    return diccionario


#Se usa para el rubro
def obtenerDatosDiccionario(pattern, diccionario_rubro):
    if diccionario_rubro is not None:
        for key, value in diccionario_rubro.items():
            if re.search(pattern, key):
                return value
    return "Sin dato"

In [ ]:
def obtenerTramos(parrafos,patternIni,patternFin):
    matching_indexIni = next((index for index, string in enumerate(reversed(parrafos), start=1) if re.search(patternIni, string)), None)
    if matching_indexIni is not None:
            matching_indexIni = len(parrafos) - matching_indexIni
            #matching_indexFin = next((index for index, string in enumerate(parrafos) if re.search(patternFin, string)), None)
            matching_indexFin = next((index for index, string in enumerate(parrafos[matching_indexIni:], start=matching_indexIni) if re.search(patternFin, string)), None)
            if matching_indexFin is not None:
                content = [paragraph for paragraph in parrafos[matching_indexIni:matching_indexFin]]
                # Join the cleaned content into a single string
                result_string = '\n'.join(content)
                return result_string, matching_indexIni, matching_indexFin
            else:
                return '',0,0
    else:
        return '',0,0

def obtenerDato(parrafos,index,palabra_clave):
    for i in range(index, len(parrafos)):
      if palabra_clave in parrafos[i].lower():  # Using lower() for case-insensitive search
          parrafo_voto = parrafos[i].strip()
          return parrafo_voto
    return ''

def obtenerDatoRegex(regex,tramos):
    matching_indexFin = next((index for index, string in enumerate(tramos) if re.search(regex, string)), None)
    if matching_indexFin is not None:
      #print(f"Index: {matching_indexFin}, Content: {parrafos[matching_indexFin]}")
      content = tramos[matching_indexFin]
              # Join the cleaned content into a single string
      #result_string = '\n'.join(content)
      return content
    else:
      return "No disponible para esta demo"
    #return next((sentence for sentence in tramos if re.search(regex, sentence)), None)

In [ ]:
def obtenerFecha (encabezado_limpio):
    #Extrae la fecha utilizando regex
    #encabezado_limpio = encabezado_limpio.replace('\n', ' ')
    fecha = ""
    if ',' in encabezado_limpio or ';' in encabezado_limpio:
      partes = re.split(r',|;',encabezado_limpio)
      for element in partes:
          #print(element)
          if "mil" in element.lower() and not 'iudad' in element:
              fecha = element
              #print("Tiene mil")
              return fecha
      for element in partes:
          if re.search(r'\d', element):
              fecha = element
              #print("Tiene numero")
              return fecha
    return encabezado_limpio

In [ ]:
def obtenerVotos(parrafos,index):
    for i in range(index, len(parrafos)):
      if "voto" in parrafos[i].lower() or "unanimidad" in parrafos[i].lower() or "mayoría" in parrafos[i].lower():  # Using lower() for case-insensitive search
          parrafo_voto = parrafos[i].strip()
          return parrafo_voto
    return ''

def sentidoVoto(parrafo_voto):
    pattern_u = re.compile(r'.*u\s*n\s*a\s*n\s*i\s*m\s*i\s*d\s*a\s*d', re.IGNORECASE)
    pattern_m = re.compile(r'.*m\s*a\s*y\s*o\s*r\s*í\s*a', re.IGNORECASE)
    if pattern_u.search(parrafo_voto.lower()):
        return 'U'
    elif pattern_m.search(parrafo_voto.lower()):
        return 'M'
    else:
        return ''


In [ ]:
def limpiar_parrafos(raw_text):
    texto_sinesp = re.sub(r'\n', ' ', raw_text)
    texto_limpio = re.sub(r'\s{3,}', '\n', texto_sinesp)
    return texto_limpio

# Pipeline

In [ ]:
def obtenerTodo(full_path):
    forma_voto = asunto = expe = preambulo = actor = resp = tercero = ponente = resolutivos = parrafo_voto = resultando = considerando =  ''
    extracted_text = extract_text_from_pdf(full_path)
    #texto_sinesp = re.sub(r'\n', ' ', extracted_text)
    #texto_limpio = re.sub(r'\s{3,}', '\n', texto_sinesp)
    parrafos = extracted_text.split('\n')
    #imprimirLista(parrafos)
    rubro=obtenerRubro(cdmxpattern,parrafos)
    if rubro == [0, 0, 0]:
        forma_voto = asunto = expe = fecha = actor = resp = tercero = ponente = resolutivos = parrafo_voto = resultando = considerando = ''
        return forma_voto, asunto, expe, fecha, actor, resp, tercero, ponente, resolutivos, parrafo_voto, resultando, considerando
    else:
         diccionario = convertirADicc(rubro)
         expe = obtenerDatosDiccionario(expe_pattern,diccionario)
         actor = obtenerDatosDiccionario(actor_pattern, diccionario)
#         resp = obtenerDatosDiccionario(resp_pattern, diccionario)
#         ponente = obtenerDatosDiccionario(pnte_pattern, diccionario)
#         #secretario = obtenerDatosDiccionario(secr_pattern, diccionario)
         tercero = obtenerDatosDiccionario(terc_pattern, diccionario)
         encabezado = obtenerDatoRegex(cdmxpattern,parrafos)
         fecha = obtenerFecha(encabezado)
#         match = TEPJF_pattern.match(expe)
#         if match:
#             asunto = match.group()  # Content of the second capturing group: 15
#         else:
#             asunto = ''
#         #Obtiene resolutivos
    resolutivos,ini,fin = obtenerTramos(parrafos,patternResuelve,patternNotif)
    resolutivos = limpiar_parrafos(resolutivos)
    if fin == 0:
        parrafo_voto = ''
        forma_voto = ''
    else:
        parrafo_voto = obtenerVotos(parrafos,fin)
        forma_voto = sentidoVoto(parrafo_voto)
    resultando, x, y = obtenerTramos(parrafos,patternResultando,patternConsiderando)
    resultando = limpiar_parrafos(resultando)
    considerando, x, y = obtenerTramos(parrafos,patternConsiderando,patternResuelve)
    considerando = limpiar_parrafos(considerando)
#  #       if 'Pleno' in

    return forma_voto,asunto,expe,fecha,actor,resp,tercero,ponente,resolutivos,parrafo_voto, resultando, considerando

filename = '399-22-EPI-01-2.pdf'
full_path = os.path.join(extracted_path, filename)
forma_voto,asunto,expe,fecha,actor,resp,tercero,ponente,resolutivos,parrafo_voto, resultando, considerando = obtenerTodo(full_path)
print(f"Expediente: {expe}")
print(f"Actor: {actor}")
#print(f"Responsable: {resp}")
#print(f"Ponente: {ponente}")
#print(f"Secretario: {secretario}")
print(f"Tercero: {tercero}")
print(f"Fecha: {fecha}") #párrafo fecha
#print(f"Asunto: {asunto}")
#print(f"Resultando: {resultando}")
print(f"Considerando: {considerando}")
print(f"Resolutivos: {resolutivos}")
#print(f"Parrafo voto: {parrafo_voto}")


In [ ]:
#TODO

data = []

for filename in os.listdir(extracted_path):
    full_path = os.path.join(extracted_path, filename)
    sentences = ""
    # Check if the current item is a file
    if os.path.isfile(full_path):
        print(filename)
        forma_voto,asunto,expe,fecha,actor,resp,tercero,ponente,resolutivos,parrafo_voto, resultando, considerando = obtenerTodo(full_path)
        sentencias = [filename,expe,actor,tercero,fecha,resultando, considerando,resolutivos]
        #sentencias = [filename,organo,forma_voto,instancia,asunto,expe,fecha,actor,resp,tercero,ponente,resolutivos,parrafo_voto]
        data.append(sentencias)

In [ ]:
# Define column names
columns = ['Filename','Expediente','Actor','Tercero','Párrafo Fecha','Resultandos','Considerandos','Puntos Resolutivos']

# Create a Pandas DataFrame
df = pd.DataFrame(data, columns=columns)
# Muestra el dataframe
df.head()

# Resultados

In [ ]:
#Revisa si hay resolutivos vacíos filename,expe,actor,tercero,preambulo,resultando, considerando,resolutivos
vac_expediente = (df['Expediente'] == '').sum()
vac_actor = (df['Actor'] == '').sum()
vac_tercero = (df['Tercero'] == '').sum()
vac_pfecha = (df['Párrafo Fecha'] == '').sum()
vac_resultandos = (df['Resultandos'] == '').sum()
vac_considerandos = (df['Considerandos'] == '').sum()
vac_resolutivos = (df['Puntos Resolutivos'] == '').sum()
vacios = [vac_expediente,vac_actor,vac_tercero,vac_pfecha,vac_resultandos,vac_considerandos,vac_resolutivos]
print(vacios)
#columns = ['Filename','Expediente','Actor','Tercero','Párrafo Fecha','Resultandos','Considerandos','Puntos Resolutivos']

In [ ]:
filtered_df = df[df['Considerandos'] == '']
filtered_df.head(20)

# Concatenación de los documentos

In [ ]:
uploaded = files.upload()

In [ ]:
df_col = pd.read_excel("Columnas (1).xlsx")

In [ ]:
df_col.head()

In [ ]:
merged_df = pd.merge(df, df_col, left_on='Filename', right_on='Título', how='inner')  # You can change 'how' parameter as needed

merged_df.head()

In [ ]:
merged_df.shape

In [ ]:
merged_df.nunique()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the figure size
plt.figure(figsize=(12, 6))

# Create the count plot
sns.countplot(x='Sentido de la Sentencia', hue='Materia', data=merged_df)

# Rotate x-axis labels for better readability (optional)
plt.xticks(rotation=45, ha='right')

# Set the title and labels
plt.title('Distribución de Materia por Sentido de la Sentencia')
plt.xlabel('Sentido de la Sentencia')
plt.ylabel('Count')

# Show the plot
plt.legend(title='Materia')  # Add legend with column name
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Calculate value counts of 'Materia' column
materia_counts = merged_df['Materia'].value_counts()

# Calculate total number of rows in DataFrame
total_rows = len(merged_df)

# Calculate percentage of each 'Materia' type
materia_percentage = (materia_counts / total_rows) * 100

# Create a pie chart
plt.figure(figsize=(10, 8))
plt.pie(materia_percentage, labels=materia_percentage.index, autopct='%1.1f%%', startangle=140)
plt.title('Porcentaje por Materia')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.show()

In [ ]:
# Calculate value counts of 'Materia' column
materia_counts = merged_df['Sentido de la Sentencia'].value_counts()

# Calculate total number of rows in DataFrame
total_rows = len(merged_df)

# Calculate percentage of each 'Materia' type
materia_percentage = (materia_counts / total_rows) * 100

# Create a pie chart
plt.figure(figsize=(10, 8))
plt.pie(materia_percentage, labels=materia_percentage.index, autopct='%1.1f%%', startangle=140)
plt.title('Porcentaje por Sentido de la Sentencia')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.show()

In [ ]:
merged_df.to_excel('sentencias_sepi.xlsx',index=False)
files.download('sentencias_sepi.xlsx')

In [ ]:
merged_df.to_json('merged_data.json', orient='records')
files.download('merged_data.json')